In [1]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
import time


In [2]:

# Load the dataset
df = pd.read_csv("crime_dataset.csv")

# Initialize geocoder
geolocator = Nominatim(user_agent="crime_heatmap_app")

# Function to get coordinates from location name
def get_coordinates(location):
    try:
        location = geolocator.geocode(f"{location}, Delhi, India")
        if location:
            return location.latitude, location.longitude
    except:
        return None, None

# Add Latitude and Longitude columns
df[['Latitude', 'Longitude']] = df['Location'].apply(lambda loc: pd.Series(get_coordinates(loc)))

# Remove entries where location couldn't be found
df.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Create the map
delhi_map = folium.Map(location=[28.6139, 77.2090], zoom_start=11)

# Add heatmap
heat_data = [[row['Latitude'], row['Longitude']] for _, row in df.iterrows()]
HeatMap(heat_data).add_to(delhi_map)

# Add legend
legend_html = '''
 <div style="
 position: fixed; 
 bottom: 50px; left: 50px; width: 150px; height: 120px; 
 background-color: white;
 border:2px solid grey; z-index:9999; font-size:14px;
 padding: 10px;
 ">
 <b>Heatmap Legend</b><br>
 <i style="background: blue; width: 10px; height: 10px; float: left; margin-right: 5px;"></i> Low Density<br>
 <i style="background: lime; width: 10px; height: 10px; float: left; margin-right: 5px;"></i> Medium Density<br>
 <i style="background: red; width: 10px; height: 10px; float: left; margin-right: 5px;"></i> High Density
 </div>
'''
delhi_map.get_root().html.add_child(folium.Element(legend_html))

# Save the map
delhi_map.save(r"C:\Users\Dell\Documents\Python\EPICS\Sign-in\templates\crime_heatmap.html")
